# Coursework 1

Author: Ziad Tamim

Date: 06/10/2023

Description: This file include coursework 1 Answers This includes:

    1- Image Transformations
    2- Convolution
    3- Video Segmentation
    4- Texture Classification
    5- Object Counting

Dataset files path for fast and easy access to files

In [ ]:
# Name in Aerial image
name_img = 'Dataset/Name in Arial.png'

# Dataset A path
face_im1 = 'Dataset/face-1.jpg'
face_im2 = 'Dataset/face-2.jpg'
face_im3 = 'Dataset/face-3.jpg'
car_im1 = 'Dataset/car-1.jpg'
car_im2 = 'Dataset/car-2.jpg'
car_im3 = 'Dataset/car-3.jpg'

test_face = 'Dataset/Test_face_image.jpg'

# Video files path
# dataset_b = 'Dataset/DatasetB.avi'
# dataset_c ='Dataset/DatasetC.avi'


: 

In [ ]:
# !pip install opencv-python

# 1) Transformations.
Rotation, translation and skew are useful operations for matching, tracking, and data augmentation.

1.a)  Write a function that takes as input an image I, rotates it by an angle θ1 and horizontally skews it by
an angle, θ2. Write the matrix formulation for image rotation R(.) and skewing S(.). Define all the
variables. Note that the origin of the coordinate system of the programming environment you use
might be different from the one shown in the lectures.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def rotate_and_skew(image_path, theta1, theta2):
    # Load the image
    image = cv2.imread(image_path)
    rows, cols, _ = image.shape

    # Compute the rotation matrix
    R = np.array([[np.cos(theta1), -np.sin(theta1)],
                  [np.sin(theta1), np.cos(theta1)]])

    # Compute the skew matrix (for horizontal skewing)
    S = np.array([[1, np.tan(theta2)],
                  [0, 1]])

    # Computing the transformation matrix T = R * S
    T = np.dot(R, S)

    # Initialize an empty image for the output
    output_image = np.zeros_like(image)

    # Compute the inverse of T for backward mapping
    T_inv = np.linalg.inv(T)

    # Iterate over the destination image (output_image)
    for y in range(rows):
        for x in range(cols):
            # Backward mapping: Find the source position for each destination pixel
            src_pos = np.dot(T_inv, np.array([x - cols/2, y - rows/2]))  # Subtract half of dimensions for center-based rotation
            src_x, src_y = src_pos[0] + cols/2, src_pos[1] + rows/2  # Add them back after transformation

            # Bilinear interpolation could be added here for better results, but for simplicity, we'll use nearest-neighbor:
            if 0 <= src_x < cols and 0 <= src_y < rows:
                output_image[y, x] = image[int(src_y), int(src_x)]

    return output_image


1.b) Create an image that contains your name written in Arial, point 72, capital letters. Rotate clockwise
the image you created by 30, 60, 120 and -50 degrees. Skew the same image by 10, 40 and 60
degrees. Complete the process so that all the pixels have a value. Discuss in the report the
advantages and disadvantages of different approaches.


In [ ]:
# Chose Roration and Translation
theta_1 = 20  # Rotation
theta_2 = 30 # Skew

# Use the function
image_path = name_img
result_image = rotate_and_skew(image_path, np.radians(theta_1), np.radians(theta_2))

# Plotting the result image
plt.imshow(result_image)
plt.axis('off')
plt.show()

1.c) Analyse the results when you change the order of the two operators: R(S(I)) and S(R(I)).

1.c.i) Rotate the image by θ1 = 20 clockwise and then skew the result by θ2 = 50.

1.c.ii) Skew the image by θ2 = 50 and then rotate the result by θ1 = 20 clockwise.

Are the results of (i) and (ii) the same? Why?

ans)
 When changing the order of the rotation (R) and the skew (S) matrix operations on the image the results of the output differ, as seen in the figures above. This result comes from computing the transformation matrix T = R * S when the order of the rotation and skewing matrix change the results differ, figure of code is seen below. This is because it involves taking the row elements of the first matrix and column elements of the second matrix, multiplying them together, and summing them up. As a result, the order and placement of the matrices heavily influence how the multiplication turns out.

# 2) Convolution.
Convolution provides a way of multiplying two arrays to produce a third array. Depending on the designed
filter and the intended effect, the kernel can be a matrix of dimensions, for example, 3x3, 5x5 or 7x7.

a) Code a function that takes an input image, performs convolution with a given kernel, and returns the
resulting image.

In [ ]:
def convolve2D(image, kernel):
    # Retrieve the dimensions of the image and the kernel
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape

    # Calculate the amount of padding needed
    pad_height = kernel_height // 2
    pad_width = kernel_width // 2

    # Calculate new image dimensions
    new_height = image_height + (2 * pad_height)
    new_width = image_width + (2 * pad_width)

    # Create a new image with zeros for padding
    padded_image = np.zeros((new_height, new_width))

    # Copy the contents of the image into the center of this new padded image
    padded_image[pad_height:pad_height + image_height, pad_width:pad_width + image_width] = image

    # Prepare an output image of the same size as the input
    output = np.zeros_like(image)

    # Perform convolution
    for y in range(image_height):
        for x in range(image_width):
            # Element-wise multiplication of the kernel and the corresponding image area and sum
            output[y, x] = np.sum(kernel * padded_image[y:y + kernel_height, x:x + kernel_width])

    return output


b) Design a convolution kernel that computes, for each pixel, the average intensity value in a 3x3 region.
Use this kernel and the filtering function above, and save the resulting image.


In [ ]:
# Average filter
kernel_ave = np.array([[1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1]]) /9

# Load image
image_path = car_im1
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Appling convolution
convolved_image = convolve2D(image, kernel_ave)

plt.figure(figsize=(10, 8))

# Original image ploted
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.subplot(1,2,1)
plt.imshow(img)
plt.title('Original image')
plt.axis('off')

# Avrage filter on image plotted
plt.subplot(1,2,2)
plt.imshow(convolved_image, cmap='gray')
plt.title('Avrage filter (Gray Scale)')
plt.axis('off')
plt.show()

Use the kernels provided below, apply the filtering function and save the resulting images. Comment
on the effect of each kernel.

kernel A

1 2 1

2 4 2

1 2 1


kernel B

0 1 0

1 -4 1

0 1 0

In [ ]:
kernel_A = np.array([[1, 2, 1],
                    [2, 4, 2],
                    [1, 2, 1]]) /16

kernel_B = np.array([[0, 1, 0],
                    [1, -4, 1],
                    [0, 1, 0]])


# Kernel A Applied
convolved_image_A = convolve2D(image, kernel_A)

# Kernel B Applied
convolved_image_B = convolve2D(image, kernel_B)

# Adjust figsize here
plt.figure(figsize=(10, 8))

# Plotting convolution with given Kernal A
plt.subplot(1,2,1)
plt.imshow(convolved_image_A, cmap='gray')
plt.title('Kernal A')
plt.axis('off')

# Plotting convolution with given Kernal A
plt.subplot(1,2,2)
plt.imshow(convolved_image_B, cmap='gray')
plt.title('Kernal B')
plt.axis('off')

plt.show()

d) Use the filtering function for the following filtering operations: (i) A followed by A; (ii) A followed by B;
(iii) B followed by A. Comment the results

In [ ]:
# Assuming image is your input image
# Applying A followed by A
image_AA = convolve2D(convolve2D(image, kernel_A), kernel_A)

# Applying A followed by B
image_AB = convolve2D(convolve2D(image, kernel_A), kernel_B)

# Applying B followed by A
image_BA = convolve2D(convolve2D(image, kernel_B), kernel_A)

# Adjust figsize here
plt.figure(figsize=(20, 10))  # Width = 10 inches, Height = 8 inches, adjust as needed

# Plotting convolution with given Kernal A
plt.subplot(3,1,1)
plt.imshow(image_AA, cmap='gray')
plt.title('Kernal A A')
plt.axis('off')

# Plotting convolution with given Kernal A
plt.subplot(3,1,2)
plt.imshow(image_AB, cmap='gray')
plt.title('Kernal A B')
plt.axis('off')

# Plotting convolution with given Kernal A
plt.subplot(3,1,3)
plt.imshow(image_BA, cmap='gray')
plt.title('Kernal B A')
plt.axis('off')

plt.show()


Comments:


# 3) Video Segmentation.
A colour histogram h(.) can be generated by counting how many times each colour occurs in an image.
Histogram intersection can be used to match a pair of histograms. Given a pair of histograms, e.g., of an
input image I and a model M, each containing n bins, the intersection of the histograms is defined as
∑ min[h(Ij), h(Mj)]
n
j=1
.

a)  Write a histogram function that returns the colour histogram of an input image. Visualize the histogram
and save the corresponding figure. For a given video sequence, use the above function to construct
the histogram of each frame.

In [ ]:
# calculate_histogram function description this function calculates the bins
# for each color RGB image and returnes the histogram for R, G and B colour
def calculate_histogram(image, bins):
    # Initialize histograms for R, G, B
    histogram_r = np.zeros(bins)
    histogram_g = np.zeros(bins)
    histogram_b = np.zeros(bins)

    # Calculate bin width
    bin_width = 256 / bins

    # Iterate through each pixel and update histograms
    for row in image:
        for pixel in row:
            r, g, b = pixel
            histogram_r[int(r // bin_width)] += 1
            histogram_g[int(g // bin_width)] += 1
            histogram_b[int(b // bin_width)] += 1

    return histogram_r, histogram_g, histogram_b

# visualize_and_save_histogram function takes in the Histogram of each image plots them in a bar chart then saves the histogram as .png
def visualize_and_save_histogram(histogram_r, histogram_g, histogram_b, bins):
    # Define bin edges
    bin_edges = np.arange(bins + 1) * (256 / bins) - 0.5

    # Plotting
    plt.figure(figsize=(10, 4))
    plt.bar(bin_edges[:-1], histogram_r, width=256/bins, color='red', alpha=0.7, label='Red Channel')
    plt.bar(bin_edges[:-1], histogram_g, width=256/bins, color='green', alpha=0.7, label='Green Channel')
    plt.bar(bin_edges[:-1], histogram_b, width=256/bins, color='blue', alpha=0.7, label='Blue Channel')
    plt.legend()
    plt.title("Color Histogram")
    plt.xlabel("Color value")
    plt.ylabel("Frequency")
    plt.xlim([-0.5, 255.5])
    plt.grid(axis='y', alpha=0.75)

    # Save plot
    # plt.savefig("color_histogram.png")
    plt.show()


In [ ]:
## Test histogram function and visualization function on video sequencs
Bin_num = 12 # Set bin number

def frames_hist(video):
    count = 0
    success = True
    while count < 347:  # To view all frames remove count < 50 to  while success.
        success,frame = video.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Change frame
        plt.imshow(frame)
        plt.title('Frame number: {}'.format(count))
        plt.axis('off')
        histogram_r, histogram_g, histogram_b = calculate_histogram(frame, Bin_num) # calculate bins for each R G B color
        visualize_and_save_histogram(histogram_r, histogram_g, histogram_b, Bin_num) # Visualize the histogram
        count += 1
        print()
    return count


video = cv2.VideoCapture(dataset_b)
frame_count = frames_hist(video)


b) Write a function that returns the value of the intersection of a pair of histograms. For a given video
sequence, use the histogram intersection function to calculate the intersection between consecutive
frames (e.g. between It and It+1, between It+1 and It+2 and so on). Find how to normalize the
intersection. Does that change the results? Plot the intersection values over time and the normalised
intersection values, and save the corresponding figures. Show and comment the figures in the report.

In [ ]:
# Function to calculate the histogram intersection
def histogram_intersection(h1, h2):
    return np.minimum(h1, h2).sum()

# Function to calculate the normalized histogram intersection
def normalized_histogram_intersection(h1, h2):
    return histogram_intersection(h1, h2) / h1.sum()

# Function to process the video and calculate histograms for each frame
def calculate_frame_histograms(video_path, bins, max_frames=None):
    video = cv2.VideoCapture(video_path)
    frame_histograms = []
    success, frame = video.read()
    frame_count = 0

    while success and (max_frames is None or frame_count < max_frames):
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        hist = calculate_histogram(frame, bins)
        frame_histograms.append(hist)
        success, frame = video.read()
        frame_count += 1

    video.release()
    return frame_histograms

# Function to plot and save histogram intersections
def plot_and_save_histogram_intersections(frame_histograms, bins):
    intersections = []
    normalized_intersections = []
    count = 0
    # Calculate intersections
    for i in range(len(frame_histograms) - 1):
        h1 = np.hstack(frame_histograms[i])
        h2 = np.hstack(frame_histograms[i + 1])
        intersection = histogram_intersection(h1, h2)
        print("The intersection result between frames {} and {} is {}".format(count, count + 1, intersection))
        normalized_intersection = normalized_histogram_intersection(h1, h2)
        print("The normalized intersection result between frames {} and {} is {}\n".format(count, count + 1, normalized_intersection))
        intersections.append(intersection)
        normalized_intersections.append(normalized_intersection)
        count += 1

    # Plotting
    plt.figure(figsize=(14, 7))
    plt.subplot(2, 1, 1)
    plt.plot(intersections, label='Intersection')
    plt.title('Histogram Intersections over Time')
    plt.xlabel('Frame number')
    plt.ylabel('Intersection value')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(normalized_intersections, label='Normalized Intersection', color='red')
    plt.title('Normalized Histogram Intersections over Time')
    plt.xlabel('Frame number')
    plt.ylabel('Normalized Intersection value')
    plt.legend()

    plt.tight_layout()
    plt.show()
    # plt.savefig("histogram_intersections.png")

# Main routine
video_path = dataset_b  # Replace with your video path
bins = 16
max_frames = None # You can set this to None to process the whole video

# Calculate histograms for each frame
frame_histograms = calculate_frame_histograms(video_path, bins, max_frames)

# Plot and save histogram intersections
plot_and_save_histogram_intersections(frame_histograms, bins)

c) Discuss in the report the following: What does the intersection value represent for a given input video?
Can you use it to make a decision about scene changes? How robust to changes in the video is the
histogram intersection? When does it fail?

# 4) Texture Classification.
The Local Binary Pattern (LBP) operator describes the surroundings of a pixel by generating a bit-code
from the binary derivatives of a pixel.

a) Write a function that divides a greyscale image into equally sized non-overlapping windows and
returns the feature descriptor for each window as distribution of LBP codes. For each pixel in the
window, compare the pixel to each of its 8 neighbours. Convert the resulting bit-codes (base 2) to
decimals (base 10 numbers) and compute their histogram over the window. Normalize the histogram
(which is now a feature descriptor representing the window). Show in the report the resulting images.

In [ ]:
# compute LBP
def get_lbp(window):
    center = window[1, 1]
    top_left = window[0, 0] >= center
    top_up = window[0, 1] >= center
    top_right = window[0, 2] >= center
    right = window[1, 2] >= center
    bottom_right = window[2, 2] >= center
    bottom_down = window[2, 1] >= center
    bottom_left = window[2, 0] >= center
    left = window[1, 0] >= center
    # Convert the booleans to binary numbers, then to the corresponding LBP
    return (top_left << 7) | (top_up << 6) | (top_right << 5) | (right << 4) | \
           (bottom_right << 3) | (bottom_down << 2) | (bottom_left << 1) | left

def lbp_feature_descriptor(image, window_size):
    # Divide the image into non-overlapping windows of specified size
    h, w = image.shape
    windows = []
    lbp_images = []
    histograms = []

    # Define number of windows along x and y axes
    num_windows_x = w // window_size
    num_windows_y = h // window_size

    for x in range(0, num_windows_x):
        for y in range(0, num_windows_y):
            # Extract the window from the image
            window = image[y*window_size:(y+1)*window_size, x*window_size:(x+1)*window_size]
            windows.append(window)

            # Compute LBP for the window
            lbp_window = np.zeros_like(window)
            for i in range(1, window_size - 1):
                for j in range(1, window_size - 1):
                    lbp_window[i, j] = get_lbp(window[i-1:i+2, j-1:j+2])

            lbp_images.append(lbp_window)

            # Compute histogram of LBP values
            hist, _ = np.histogram(lbp_window.ravel(), bins=np.arange(0, 256), density=True)
            histograms.append(hist)

    return windows, lbp_images, histograms

# Gray scale image
image = cv2.imread(car_im1, cv2.IMREAD_GRAYSCALE)

# Define window size
window_size = 80

# Get LBP feature descriptors
windows, lbp_images, histograms = lbp_feature_descriptor(image, window_size)

# Display window, lbp image and histograms 
indices_to_display = [0, len(windows)//2, len(windows)-1]  # non-consecutive indices
for index in indices_to_display:
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.imshow(windows[index], cmap='gray')
    plt.title(f'Window {index+1}')

    plt.subplot(1, 3, 2)
    plt.imshow(lbp_images[index], cmap='gray')
    plt.title(f'LBP Window {index+1}')

    plt.subplot(1, 3, 3)
    plt.bar(np.arange(len(histograms[index])), histograms[index])
    plt.title(f'Histogram {index+1}')
    plt.tight_layout()
    plt.show()


b) Come up with a descriptor that represents the whole image as consisting of multiple windows. For
example, you could combine several local descriptions into a global description by concatenation.
Discuss in the report alternative approaches. Using the global descriptor you created, implement a
classification process that separates the images in the dataset into two categories: face images and
non-face images (for example, you could use histogram similarities). Comment the results in the
report. Is the global descriptor able to represent whole images of different types (e.g. faces vs. cars)?
Identify problems (if any), discuss them in the report and suggest possible solutions.


In [ ]:
from skimage.io import imread
from scipy.stats import chisquare

def plot_global_descriptor(descriptor, title):
    # Plot the global descriptor as a bar plot
    plt.figure(figsize=(10, 4))
    plt.bar(range(len(descriptor)), descriptor, width=1)
    plt.title(f'Global Descriptor for {title}')
    plt.xlabel('LBP Code')
    plt.ylabel('Normalized Frequency')
    plt.show()

def compute_global_descriptor(image_path, window_size=50): # Change window size from here
    # Read the grayscale image
    image = imread(image_path, as_gray=True)

    # Compute LBP for each window and get histograms
    _, _, histograms = lbp_feature_descriptor(image, window_size)

    # Concatenate all histograms to form a global descriptor
    global_descriptor = np.concatenate(histograms)
    return global_descriptor

# Paths to the images
face_image_path = face_im1 # image 1 path
car_image_path = car_im1  # image 2 path

# Compute global descriptors for the given images
face_descriptor = compute_global_descriptor(face_image_path)
car_descriptor = compute_global_descriptor(car_image_path)

# Plot the global descriptors
plot_global_descriptor(face_descriptor, 'Face Image')
plot_global_descriptor(car_descriptor, 'Car Image')

In [ ]:
def chi_squared_distance(hist1, hist2):
    # Calculate the Chi-Squared distance between two histograms.
    return np.sum((hist1 - hist2) ** 2 / (hist1 + hist2 + 1e-10))

def classify_image(test_descriptor, reference_descriptors, labels):
    # Classify an image as face or non-face based on histogram similarity.
    distances = [chi_squared_distance(test_descriptor, ref) for ref in reference_descriptors]
    closest_index = np.argmin(distances)
    return labels[closest_index]

# faces and non-faces path
face_images = [
    face_im1,
    face_im2,
    face_im3
]

non_face_images = [
    car_im1,
    car_im2,
    car_im3
]

# Compute global descriptors for the training dataset
face_descriptors = [compute_global_descriptor(face_image) for face_image in face_images]  # List of face image paths
non_face_descriptors = [compute_global_descriptor(non_face_image) for non_face_image in non_face_images]  # List of non-face image paths

# Combine descriptors and labels
all_descriptors = face_descriptors + non_face_descriptors
labels = ['face'] * len(face_descriptors) + ['non-face'] * len(non_face_descriptors)

# Example classification of a new image
test_image_path = test_face  # Replace with test image car_im1 or test_face
test_descriptor = compute_global_descriptor(test_image_path)

classification = classify_image(test_descriptor, all_descriptors, labels)
print(f"The image is classified as: {classification}")


c) Decrease the window size and perform classification again. Comment the results in the report.

Comment in the report

d) Increase the window size and perform classification again. Comment the results in the report.

Comment in the report

e) Discuss how LBP can be used or modified for the analysis of dynamic textures in a video.

Comment in the report

# 5) Object Counting.
Moving objects captured by fixed cameras are the focus of several computer vision applications.

a) Write a function that performs pixel-by-pixel frame differencing using, as reference frame, the first
frame of an image sequence. Apply a classification threshold and save the results.


In [ ]:
def frame_differencing(video_path, threshold_value, frame_indices):
    """
    Performs frame differencing on a video sequence

    Parameters:
    - video_path: Path to the video file
    - threshold_value: Value for classification threshold.
    - frame_indices: A list of frame indices to compare with the first frame

    Output:
    - Plots original frames, frame differencing results, and threshold results
    - Saves results
    """

    # Open video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file.")
        return

    # Read the first frame
    ret, reference_frame = cap.read()
    if not ret:
        print("Error reading the first frame.")
        return

    reference_frame_gray = cv2.cvtColor(reference_frame, cv2.COLOR_BGR2GRAY)

    # Plot the reference frame
    plt.figure(figsize=(12, 8))
    plt.subplot(3, 3, 1)
    plt.imshow(reference_frame_gray, cmap='gray')
    plt.title('Reference Frame')
    plt.axis('off')

    # Go through the selected frames
    for i, frame_index in enumerate(frame_indices, start=1):
        # Set the video frame position
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)

        # Read the selected frame
        ret, selected_frame = cap.read()
        if not ret:
            print(f"Error reading frame at index {frame_index}.")
            continue

        selected_frame_gray = cv2.cvtColor(selected_frame, cv2.COLOR_BGR2GRAY)

        # Compute the absolute difference using numpy to avoid using cv2 in the main algorithm
        frame_diff = np.abs(reference_frame_gray.astype(np.int16) - selected_frame_gray.astype(np.int16)).astype(np.uint8)

        # Apply threshold using numpy to avoid using cv2 in the main algorithm
        thresh_img = np.where(frame_diff > threshold_value, 255, 0).astype(np.uint8)

        # Save results using plt.imsave to avoid using cv2
        #plt.imsave(f'frame_diff_{frame_index}.png', frame_diff, cmap='gray')
        #plt.imsave(f'thresh_{frame_index}.png', thresh_img, cmap='gray')

        # Plot original frames
        plt.subplot(3, 3, i+1)
        plt.imshow(selected_frame_gray, cmap='gray')
        plt.title(f'Selected Frame {i}')
        plt.axis('off')

        # Plot frame differencing results
        plt.subplot(3, 3, i+4)
        plt.imshow(frame_diff, cmap='gray')
        plt.title(f'Frame Differencing {i}')
        plt.axis('off')

        # Plot threshold results
        plt.subplot(3, 3, i+7)
        plt.imshow(thresh_img, cmap='gray')
        plt.title(f'Threshold Result {i}')
        plt.axis('off')

    plt.show()

    # Release video capture object
    cap.release()

# Define video path and parameters
video_path = dataset_c
threshold_value = 50 # Set threshold value
frame_indices = [10, 30] # Set the frame indices you want to compare

# Run the updated frame differencing function
frame_differencing(video_path, threshold_value, frame_indices)


b) Repeat the exercise using the previous frame as reference frame (use frame It-1 as reference frame
for frame It, for each t). Comment the results in the report.

In [ ]:
def frame_differencing_previous(video_path, threshold_value, frame_indices):
    """
    Performs frame differencing using the previous frame as the reference frame.

    Parameters:
    - video_path: Path to the video file
    - threshold_value: Value for classification threshold.
    - frame_indices: A list of frame indices to compare with the previous frame

    Output:
    - Plots original frames, frame differencing results, and threshold results
    - Saves results
    """

    # Open video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file.")
        return

    # Initialize plot
    fig, axes = plt.subplots(3, 2, figsize=(12, 8))

    # Go through the selected frames
    for i, frame_index in enumerate(frame_indices):
        # Read the previous frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index-1)
        ret, prev_frame = cap.read()
        if not ret:
            print(f"Error reading frame at index {frame_index-1}.")
            continue

        # Convert previous frame to grayscale
        prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

        # Read the current frame
        ret, current_frame = cap.read()
        if not ret:
            print(f"Error reading frame at index {frame_index}.")
            continue

        # Convert current frame to grayscale
        current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

        # Compute the absolute difference using numpy
        frame_diff = np.abs(prev_frame_gray.astype(np.int16) - current_frame_gray.astype(np.int16)).astype(np.uint8)

        # Apply threshold using numpy
        thresh_img = np.where(frame_diff > threshold_value, 255, 0).astype(np.uint8)

        # Save results
        #plt.imsave(f'frame_diff_{frame_index}.png', frame_diff, cmap='gray')
        #plt.imsave(f'thresh_{frame_index}.png', thresh_img, cmap='gray')

        # Plot original frame
        axes[0, i].imshow(current_frame_gray, cmap='gray')
        axes[0, i].set_title(f'Selected Frame {frame_index}')
        axes[0, i].axis('off')

        # Plot frame differencing result
        axes[1, i].imshow(frame_diff, cmap='gray')
        axes[1, i].set_title(f'Frame Differencing {frame_index}')
        axes[1, i].axis('off')

        # Plot threshold result
        axes[2, i].imshow(thresh_img, cmap='gray')
        axes[2, i].set_title(f'Threshold Result {frame_index}')
        axes[2, i].axis('off')

    plt.tight_layout()
    plt.show()

    # Release video capture object
    cap.release()

# Define video path and parameters
video_path = dataset_c  
threshold_value = 50  # Set threshold value
frame_indices = [5, 20]  # Set the frame indices


frame_differencing_previous(video_path, threshold_value, frame_indices)



c) Write a function that generates a reference frame (background) for the sequence using for example
frame differencing and a weighted temporal averaging algorithm.

In [ ]:
def generate_background(video_path, alpha):
    """
    Generates a reference background frame for a video sequence using weighted temporal averaging.

    Parameters:
    - video_path
    - alpha: Weight for the running average

    Output:
    - Plots the generated background
    """

    # Open video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file.")
        return None

    # Read the first frame to initialize the background
    ret, frame = cap.read()


    # Initialize the background with the first frame
    background = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.float32)

    # Process the video frames
    while True:
        # Read the next frame
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        # Convert frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.float32)

        # Update the background using a weighted average (manual implementation)
        background = (1 - alpha) * background + alpha * gray_frame

    # Convert the floating background to uint8 for displaying
    background_uint8 = np.clip(background, 0, 255).astype(np.uint8)

    # Plot the generated background
    plt.figure(figsize=(6, 6))
    plt.imshow(background_uint8, cmap='gray')
    plt.title('Generated Background')
    plt.axis('off')
    plt.show()

    # Release video capture object
    cap.release()

    return background_uint8

# Video path
video_path = dataset_c
alpha = 0.04  # Alpha for weighted average

background = generate_background(video_path, alpha)

d) Write a function that counts the number of moving objects in each frame of a sequence. Generate a
bar plot that visualizes the number of objects for each frame of the whole sequence. Discuss in the
report the implemented solution, including advantages and disadvantages.

In [ ]:
from scipy import ndimage

def count_moving_objects(video_path, background, threshold_value, min_contour_area=250):
    """
    Counts the number of moving objects in each frame of a video sequence.

    Parameters:
    - video_path
    - background: The background frame for reference used in question 5 c
    - threshold_value
    - min_contour_area: Minimum area for a contour to be considered an object

    Output:
    - A bar plot of the number of moving objects per frame
    """

    # Open video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file.")
        return

    frame_counts = []
    frame_index = 0

    while True:
        # Read the next frame
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        # Convert frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Compute the absolute difference with the background
        frame_diff = np.abs(gray_frame.astype(np.int16) - background.astype(np.int16)).astype(np.uint8)

        # Apply threshold
        thresh = np.where(frame_diff > threshold_value, 255, 0).astype(np.uint8)

        # Apply morphological operations to reduce noise
        kernel = np.ones((5,5),np.uint8)
        erosion = ndimage.binary_erosion(thresh, structure=kernel).astype(np.uint8)
        dilation = ndimage.binary_dilation(erosion, structure=kernel).astype(np.uint8)

        # Find contours using connected components
        labeled_array, num_features = ndimage.label(dilation)

        # Measure the size of each contour and filter out very small contours
        object_slices = ndimage.find_objects(labeled_array)
        moving_objects = [s for s in object_slices if np.product(labeled_array[s].shape) > min_contour_area]

        # Count the number of moving objects
        object_count = len(moving_objects)
        frame_counts.append(object_count)

        frame_index += 1

    # Release video capture object
    cap.release()

    # Plot the bar plot of moving objects per frame
    plt.figure(figsize=(10, 5))
    plt.bar(range(len(frame_counts)), frame_counts)
    plt.xlabel('Frame')
    plt.ylabel('Number of Moving cars')
    plt.title('Number of Moving cars Per Frame')
    plt.show()

    return frame_counts

# Define video path and threshold value
video_path = dataset_c  # video path
threshold_value = 30  # Threshold value

# Execute function
moving_objects_counts = count_moving_objects(video_path, background, threshold_value) # Background used from previous question
